In [ ]:
import pandas as pd
from MCForecastTools import MCSimulation
import requests
from dotenv import load_dotenv
import os
import numpy as np
import seaborn as sns
import yfinance as yf
import panel as pn
from panel.interact import interact
from panel import widgets
import plotly.express as px
pn.extension('plotly')

from scipy import optimize
import datetime
import hvplot.pandas
%matplotlib inline

In [ ]:
def return_on_lending(amount, interest_rate, duration):
    def xnpv(rate, cashflows):
        return sum([cf/(1+rate)**((t-cashflows[0][0]).days/365.0) for (t,cf) in cashflows])
 
    def xirr(cashflows, guess=0.1):
        try:
            return optimize.newton(lambda r: xnpv(r,cashflows),guess)
        except:
            print('Calc Wrong')

    loan_amount = amount
    rate = interest_rate
    initial_investment = -loan_amount
    monthly_Payment = loan_amount * rate * (30/360)
    last_payment = loan_amount + monthly_Payment
    date_list =['2021-01-06', '2021-01-07', '2021-01-08', '2021-01-09', '2021-01-10', '2021-01-11', '2021-01-12', '2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06']
    ids_list = [1,2,3,4,5,6,7,8,9,10,11,12,13]
    flows_list = [ initial_investment, monthly_Payment, monthly_Payment, monthly_Payment, monthly_Payment, monthly_Payment, monthly_Payment, monthly_Payment, monthly_Payment, monthly_Payment,  monthly_Payment, monthly_Payment, last_payment]
    df = pd.DataFrame(list(zip(date_list,ids_list,flows_list)), columns=['Date','ID','Flow'])
    df['Date'] = pd.to_datetime(df['Date']).dt.date
    data = list(zip(df['Date'].values,df['Flow'].values))
    value = xirr(data)
    return round((value * amount * duration),2)

In [ ]:
# GET TICKERS
tickers = input('what do you want to invest in?').split()

In [ ]:
# GET CAPITAL AMOUNT TO BE INVESTED
capital = int(input('how much do you have to invest?'))

In [ ]:
# GET YEARLY DURATION FOR FORECASTING
duration = int(input('in years, chose a duration for the forecasting'))

In [ ]:
# GET HISTORIAL DATA FROM YAHOO FINANCE FOR ASSETS CHOSEN
df_portfolio = yf.download(tickers, start='2019-03-20', end='2020-03-20')

In [ ]:
# CREATE DATAFRAME OF JUST CLOSING PRICES
df_closing = df_portfolio['Adj Close'].dropna()

In [ ]:
# CALCULATE DAILY RETURNS
daily_returns = df_closing.pct_change().dropna()

In [ ]:
# CALCULATE CORRELATION OF ASSETS 
correlations = daily_returns.corr()

In [ ]:
# CALCULATE ANNUALIZED STANDARD DEVIATION OF ASSETS
std_deviations = daily_returns.std()
annualized_std = (std_deviations * np.sqrt(252)).sort_values(ascending=False)

In [ ]:
# CALCULATE SHARP RATIOS
sharpe_ratios = ((daily_returns.mean() * 252)) / (daily_returns.std() * np.sqrt(252))

In [ ]:
# CREATE WEIGHTS FOR PORTFOLIOS
low_risk_weights = [.05, .1, .15, .2, .5]
med_risk_weights = [.1, .15, .2, .25, .3]
high_risk_weights = [.3, .25, .2, .15, .1]

# LIST OF ASSETS IN ORDER OF MOST VOLATILE TO LEAST VOLATILE 
volatitly_ranks = annualized_std.index.to_list()

# DAILY RETURNS DATAFRAME REORDERED IN TERMS OF VOLATILITY
ordered_volatility_df_pct = pd.concat([daily_returns[volatitly_ranks[0]], daily_returns[volatitly_ranks[1]],daily_returns[volatitly_ranks[2]],daily_returns[volatitly_ranks[3]],daily_returns[volatitly_ranks[4]]], axis=1, join='inner')

#CLOSING PRICES DATAFRAME REORDERED IN TERMS OF VOLATILITY
ordered_volatility_df_closing = pd.concat([df_closing[volatitly_ranks[0]], df_closing[volatitly_ranks[1]],df_closing[volatitly_ranks[2]],df_closing[volatitly_ranks[3]],df_closing[volatitly_ranks[4]]], axis=1, join='inner')

# RISK APPETITE PORTOLIOS USING DAILY RETURNS DATA
low_risk_portfolio_pct = ordered_volatility_df_pct.dot(low_risk_weights)
med_risk_portfolio_pct = ordered_volatility_df_pct.dot(med_risk_weights)
high_risk_portfolio_pct = ordered_volatility_df_pct.dot(high_risk_weights)

# RISK APPETITE PORTOLIOS USING CLOSE PRICES DATA
low_risk_portfolio_close = ordered_volatility_df_closing.dot(low_risk_weights)
med_risk_portfolio_close = ordered_volatility_df_closing.dot(med_risk_weights)
high_risk_portfolio_close = ordered_volatility_df_closing.dot(high_risk_weights)

In [ ]:
# CALCULATE CUMULATIVE RETURNS FOR ALL PORTFOLIOS (DAILY RETURNS DATA)
low_risk_cumulative_returns = (1 + low_risk_portfolio_pct).cumprod() - 1
med_risk_cumulative_returns = (1 + med_risk_portfolio_pct).cumprod() - 1
high_risk_cumulative_returns = (1 + high_risk_portfolio_pct).cumprod() - 1

In [ ]:
# CREATE MULTI LEVEL DATAFRAME FOR MONTE CARLO SIMULATIONS
multi_level = pd.concat([ordered_volatility_df_closing], axis=1, keys=['close']).swaplevel(0,1,1)

In [ ]:
# CREATE MC INSTANCES OF LOW, MED AND HIGH WEIGHTS

MC_low_risk = MCSimulation(
    portfolio_data = multi_level,
    weights = low_risk_weights,
    num_simulation = 100,
    num_trading_days = 252*duration
)

MC_med_risk = MCSimulation(
    portfolio_data = multi_level,
    weights = med_risk_weights,
    num_simulation = 100,
    num_trading_days = 252*duration
)

MC_high_risk = MCSimulation(
    portfolio_data = multi_level,
    weights = high_risk_weights,
    num_simulation = 100,
    num_trading_days = 252*duration
)

In [ ]:
# RUN MONTE CARLO CUMULATIVE RETURN FOR LOW RISK
MC_low_risk.calc_cumulative_return()

In [ ]:
# RUN MONTE CARLO CUMULATIVE RETURN FOR MEDIUM RISK
MC_med_risk.calc_cumulative_return()

In [ ]:
# RUN MONTE CARLO CUMULATIVE RETURN FOR HIGH RISK
MC_high_risk.calc_cumulative_return()

In [ ]:
# CREATE SUMMARY STATS FOR EACH MONTE CARLO SIMULATION
low_risk_table = MC_low_risk.summarize_cumulative_return()
med_risk_table = MC_med_risk.summarize_cumulative_return()
high_risk_table = MC_high_risk.summarize_cumulative_return()

In [ ]:
# RISK/ASSET ANALYSIS VISUALIZATIONS
daily_returns_hist = daily_returns.hvplot.hist(bins=100, figsize=(25,10))
closing_prices_line = df_closing.hvplot.line(figsize=(25,10),logy=True)
correlations_heatmap = correlations.hvplot.heatmap(cmap=["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"])
std_devs_bar = annualized_std.hvplot.bar()
sharp_ratios_bar = sharpe_ratios.hvplot.bar()

# PORTFOLIO PERFORMANCE VISUALIZATIONS
low_risk_close_plot = low_risk_portfolio_close.hvplot.line(label="Low Risk").opts(show_legend=True)
med_risk_close_plot = med_risk_portfolio_close.hvplot.line(label="Medium Risk").opts(show_legend=True)
high_risk_close_plot = high_risk_portfolio_close.hvplot.line(label="High Risk").opts(show_legend=True)
portfolios_plot_close_data = (low_risk_close_plot * med_risk_close_plot * high_risk_close_plot).opts(legend_position="top_left")

low_risk_cumulative_plot = low_risk_cumulative_returns.hvplot(label="Low Risk").opts(show_legend=True)
med_risk_cumulative_plot = med_risk_cumulative_returns.hvplot(label="Medium Risk").opts(show_legend=True)
high_risk_cumulative_plot = high_risk_cumulative_returns.hvplot(label="High Risk").opts(show_legend=True)
portfolios_plot_cumulative_data = (low_risk_cumulative_plot * med_risk_cumulative_plot * high_risk_cumulative_plot).opts(legend_position="top_left")

# MONTE CARLO VISUALIZATIONS
low_returns_data = {
    "mean": list(MC_low_risk.simulated_return.mean(axis=1)),
    "median":list(MC_low_risk.simulated_return.median(axis=1)),
    "max":list(MC_low_risk.simulated_return.max(axis=1)),
    "min":list(MC_low_risk.simulated_return.min(axis=1))
}
low_return_df = pd.DataFrame(low_returns_data)
low_risk_MC_plot = low_return_df.hvplot()

medium_returns_data = {
    "mean": list(MC_med_risk.simulated_return.mean(axis=1)),
    "median":list(MC_med_risk.simulated_return.median(axis=1)),
    "max":list(MC_med_risk.simulated_return.max(axis=1)),
    "min":list(MC_med_risk.simulated_return.min(axis=1))
}
medium_return_df = pd.DataFrame(medium_returns_data)
med_risk_MC_plot = medium_return_df.hvplot()

high_returns_data = {
    "mean": list(MC_high_risk.simulated_return.mean(axis=1)),
    "median":list(MC_high_risk.simulated_return.median(axis=1)),
    "max":list(MC_high_risk.simulated_return.max(axis=1)),
    "min":list(MC_high_risk.simulated_return.min(axis=1))
}
high_return_df = pd.DataFrame(high_returns_data)
high_risk_MC_plot = high_return_df.hvplot()

In [ ]:
portfolios_plot_cumulative_data

In [ ]:
def lendingViz():
    return interact(return_on_lending, amount=(capital*.2,capital*.5), interest_rate=0.04, duration=duration )

In [ ]:
# DASHBOARD CREATION

daily_returns_col = pn.Column('## Daily Returns', daily_returns_hist)
sharp_ratios_col = pn.Column('## Sharp Ratios', sharp_ratios_bar)
daily_closing_col = pn.Column('## Closing Prices', closing_prices_line)
std_dev_col = pn.Column('## Annualized STDs', std_devs_bar)
correlations_col = pn.Column('## Correlations', correlations_heatmap)
low_risk_mc_col = pn.Column('## Monte Carlo Low Risk Portfolio', low_risk_MC_plot)
med_risk_mc_col = pn.Column('## Monte Carlo Medium Risk Portfolio', med_risk_MC_plot)
high_risk_mc_col = pn.Column('## Monte Carlo High Risk Portfolio', high_risk_MC_plot)

portfolio_close_col = pn.Column('## Portfolio Close Data', portfolios_plot_close_data)
portfolio_cumulative_col = pn.Column('## Portfolio Cumulative Returns', portfolios_plot_cumulative_data)

risk_analysis_tabs = pn.Tabs(
    ('Closing Prices', daily_closing_col),
    ('Daily Returns', daily_returns_col),
    ('Sharp Ratios', sharp_ratios_col),
    ('Standard Deviations', std_dev_col),
    ('Correlations', correlations_col)
)

lending_tab = pn.Tabs(
    ('Lending Interaction', lendingViz())
)

portfolio_performace_tab = pn.Tabs(
    ('Daily Closes', portfolio_close_col),
    ('Cumulative Returns', portfolio_cumulative_col)
)

mc_tab = pn.Tabs(
    ('Low', low_risk_mc_col),
    ('Medium', med_risk_mc_col),
    ('High', high_risk_mc_col)
)
mainTabs = pn.Tabs(
    ('Risk Analysis', risk_analysis_tabs),
    ('Performance', portfolio_performace_tab),
    ('Monte Carlo', mc_tab),
    ('Lending', lending_tab)
)
dashboard = pn.Column('# Analysis', mainTabs)

In [ ]:
dashboard

In [ ]:
 # Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $15,000 investments in stocks
def mc_and_lending_summary(capital, duration):
    
    low_risk_investment_amount = capital * .8
    low_risk_lending_amount = capital * .2
    
    med_risk_investment_amount = capital * .6
    med_risk_lending_amount = capital * .4
    
    high_risk_investment_amount = capital * .5
    high_risk_lending_amount = capital * .5
    
    low_risk_ci_lower = round(low_risk_table[8]*low_risk_investment_amount,2)
    low_risk_ci_upper = round(low_risk_table[9]*low_risk_investment_amount,2)
    med_risk_ci_lower = round(med_risk_table[8]*med_risk_investment_amount,2)
    med_risk_ci_upper = round(med_risk_table[9]*med_risk_investment_amount,2)
    high_risk_ci_lower = round(high_risk_table[8]*high_risk_investment_amount,2)
    high_risk_ci_upper = round(high_risk_table[9]*high_risk_investment_amount,2)
    
    low_risk_lending_profit = return_on_lending(low_risk_lending_amount, .04, duration)
    med_risk_lending_profit = return_on_lending(med_risk_lending_amount, .04, duration)
    high_risk_lending_profit = return_on_lending(high_risk_lending_amount, .04, duration)
    # Print results
    print(
          f"There is a 95% chance that an initial investment of ${low_risk_investment_amount} in the low risk portfolio"
          f" over the next 5 years will end within in the range of"
          f" ${low_risk_ci_lower} and ${low_risk_ci_upper}. "
          f"Additionally, lending ${low_risk_lending_amount} would yield ${low_risk_lending_profit} in profit." 
         )
    print(
          f"There is a 95% chance that an initial investment of ${med_risk_investment_amount} in the medium risk portfolio"
          f" over the next 5 years will end within in the range of"
          f" ${med_risk_ci_lower} and ${med_risk_ci_upper}. "
          f"Additionally, lending ${med_risk_lending_amount} would yield ${med_risk_lending_profit} in profit."
         )

    print(
          f"There is a 95% chance that an initial investment of ${high_risk_investment_amount} in the high risk portfolio"
          f" over the next 5 years will end within in the range of"
          f" ${high_risk_ci_lower} and ${high_risk_ci_upper}. "
          f"Additionally, lending ${high_risk_lending_amount} would yield ${high_risk_lending_profit} in profit." 
         )

In [ ]:
mc_and_lending_summary(capital, duration)